In [1]:
import numpy as np
from astropy.table import Table
import subprocess
import os
from astroquery.sdss import SDSS
from astropy.coordinates import SkyCoord
import astropy.units as u

In [2]:
#Read the full database
dbase = Table.read("dbase.fits")

In [3]:
#Set objects we want to focus on
wids = [
    "W0019-1046",
    "W0116-0505",
    "W0204-0506",
    "W0220+0137",
    "W0831+0140",
]
wids = np.array(wids, dtype=bytes)

In [4]:
#Limit the database to only the objects of interest. 
dbase = dbase[np.isin(dbase['Short Name'],wids)]

In [5]:
#Convert VHS photometry from Vega to AB using the values in http://casu.ast.cam.ac.uk/surveys-projects/vista/technical/filter-set
AB_Vega_corr = {"y":0.600,"j":0.916,"h":1.366,"ks":1.827}
for band in ["y","j","h","ks"]:
    dbase[band+"petromag (VHS DR5)"] += AB_Vega_corr[band]

In [6]:
#Convert UKIDSS from Vega to AB using the values in Table 7 of Hewett et al. (2006, MNRAS, 367, 454) 
AB_Vega_corr = {"y":0.634,"j":0.938,"h":1.379,"k":1.900}
for band in ["y","j","h","k"]:
    dbase[band+"petromag (UKIDSS DR11)"] += AB_Vega_corr[band]

In [7]:
phot = Table()
phot['WID'] = dbase['Short Name']

In [8]:
phot['zspec'] = np.where(dbase['zspec refined'].mask, dbase['zspec'], dbase['zspec refined'])
#If we have a non-optical redshift use that instead. 
phot['zspec'][~dbase['zspec_nopt'].mask] = dbase['zspec_nopt'][~dbase['zspec_nopt'].mask]

In [9]:
#Set the bands we want in the form we need them. 
cols = [
    'w1mpro (CatWISE)',
    'w1sigmpro (CatWISE)',
    'w2mpro (CatWISE)',
    'w2sigmpro (CatWISE)',
    'w3mpro',
    'w3sigmpro',
    'w4mpro',
    'w4sigmpro',
    # 'gMeanPSFMag (PS1)',
    # 'gMeanPSFMagErr (PS1)',
    # 'rMeanPSFMag (PS1)',
    # 'rMeanPSFMagErr (PS1)',
    # 'iMeanPSFMag (PS1)',
    # 'iMeanPSFMagErr (PS1)',
    # 'zMeanPSFMag (PS1)',
    # 'zMeanPSFMagErr (PS1)',
    # 'yMeanPSFMag (PS1)',
    # 'yMeanPSFMagErr',
    # 'gm (Legacy DR10)',
    # 'gm_e (Legacy DR10)',
    # 'rm (Legacy DR10)',
    # 'rm_e (Legacy DR10)',
    # 'im (Legacy DR10)',
    # 'im_e (Legacy DR10)',
    # 'zm (Legacy DR10)',
    # 'zm_e (Legacy DR10)',
    'ypetromag (VHS DR5)',
    'ypetromagerr (VHS DR5)',
    'jpetromag (VHS DR5)',
    'jpetromagerr (VHS DR5)',
    'hpetromag (VHS DR5)',
    'hpetromagerr (VHS DR5)',
    'kspetromag (VHS DR5)',
    'kspetromagerr (VHS DR5)',
    'ypetromag (UKIDSS DR11)',
    'ypetromagerr (UKIDSS DR11)',
    'jpetromag (UKIDSS DR11)',
    'jpetromagerr (UKIDSS DR11)',
    'hpetromag (UKIDSS DR11)',
    'hpetromagerr (UKIDSS DR11)',
    'kpetromag (UKIDSS DR11)',
    'kpetromagerr (UKIDSS DR11)',
    'targeted r mag',
    'targeted r magerr',
    'targeted g mag',
    'targeted g magerr',
    'targeted J mag',
    'targeted J magerr',
    'targeted H mag',
    'targeted H magerr',
    'targeted KS mag',
    'targeted KS magerr',
    'i1mag',
    'i1magerr',
    'i2mag',
    'i2magerr',
]
for col in cols:
    phot[col] = dbase[col]
    phot[col].fill_value = -99.

In [10]:
#Add SDSS photometry. 
photoobj_fields = [
    'ra',
    'dec',
    'objid',
    'run',
    'rerun',
    'camcol',
    'field',
    'u',
    'Err_u',
    'g',
    'Err_g',
    'r',
    'Err_r',
    'i',
    'Err_i',
    'z',
    'Err_z',
    'mjd',
]
bhd_c = SkyCoord(ra=dbase['ra']*u.deg, dec=dbase['dec']*u.deg)
xid = SDSS.query_crossid(bhd_c, radius='1.5 arcsec', data_release=18, photoobj_fields=photoobj_fields, spectro=False)

In [11]:
xid.show_in_notebook()

idx,name,ra,dec,objid,run,rerun,camcol,field,u,Err_u,g,Err_g,r,Err_r,i,Err_i,z,Err_z,mjd,type
0,obj_0,4.8619706504028,-10.7758643109812,1237652629100691717,1666,301,2,224,22.30112,0.383742,22.0817,0.1355587,21.98974,0.1762975,21.77306,0.2220604,21.21562,0.516012,51792,GALAXY
1,obj_1,19.0059557982554,-5.08442448812927,1237679322854326868,7881,301,3,148,23.57158,0.6852992,21.4612,0.0539205,21.38975,0.05434495,21.74447,0.09403163,21.37522,0.2567134,54830,STAR
2,obj_2,31.1920725190906,-5.1112871057984,1237679253596275430,7865,301,2,122,23.00033,0.600696,22.65626,0.1662882,22.48925,0.2340006,21.79878,0.1752908,22.03948,0.6588346,54822,GALAXY
3,obj_3,35.217145393166,1.61976337640961,1237678437551244102,7675,301,2,103,23.46645,0.5877249,21.77853,0.05916739,21.84585,0.08617316,22.06462,0.1318762,21.62065,0.2711797,54715,STAR
4,obj_4,127.971863713442,1.66970537198585,1237653665249231950,1907,301,4,36,26.25483,0.5037822,23.38177,0.3257467,21.7997,0.1305126,21.66239,0.1698755,21.45549,0.6501953,51878,GALAXY


In [12]:
#Add to the photometric table. 
for band in ["u","g","r","i","z"]:
    phot['{} (SDSS DR18)'.format(band)] = xid[band]
    phot['{}_err (SDSS DR18)'.format(band)] = xid['Err_'+band]
    cols.extend(['{} (SDSS DR18)'.format(band), '{}_err (SDSS DR18)'.format(band)])

In [13]:
#Save the photometry file. 
phot.filled().write("phot.dat", format='ascii.no_header', overwrite=True)

In [14]:
#Save the number of bands we are using. 
cato = open("nchan.txt","w")
cato.write("{}\n".format(len(cols[::2])))
cato.close()

In [15]:
filters = {
    'w1mpro (CatWISE)'       : {"bname":"real_wise_band_1","norm":1,"zp":306.681},
    'w2mpro (CatWISE)'       : {"bname":"real_wise_band_2","norm":1,"zp":170.663},
    'w3mpro'                 : {"bname":"real_wise_band_3","norm":1,"zp":29.0448},
    'w4mpro'                 : {"bname":"real_wise_band_4","norm":1,"zp":8.2839},
    'gMeanPSFMag (PS1)'      : {"bname":"gp1","norm":3,"zp":3631},
    'rMeanPSFMag (PS1)'      : {"bname":"rp1","norm":3,"zp":3631},
    'iMeanPSFMag (PS1)'      : {"bname":"ip1","norm":3,"zp":3631},
    'zMeanPSFMag (PS1)'      : {"bname":"zp1","norm":3,"zp":3631},
    'yMeanPSFMag (PS1)'      : {"bname":"yp1","norm":3,"zp":3631},
    'gm (Legacy DR10)'       : {"bname":"DECAM_g","norm":3,"zp":3631},
    'rm (Legacy DR10)'       : {"bname":"DECAM_r","norm":3,"zp":3631},
    'im (Legacy DR10)'       : {"bname":"DECAM_i","norm":3,"zp":3631},
    'zm (Legacy DR10)'       : {"bname":"DECAM_z","norm":3,"zp":3631},
    'ypetromag (VHS DR5)'    : {"bname":"vistaY" ,"norm":3,"zp":3631},
    'jpetromag (VHS DR5)'    : {"bname":"vistaJ" ,"norm":3,"zp":3631},
    'hpetromag (VHS DR5)'    : {"bname":"vistaH" ,"norm":3,"zp":3631},
    'kspetromag (VHS DR5)'   : {"bname":"vistaKs" ,"norm":3,"zp":3631},
    'ypetromag (UKIDSS DR11)': {"bname":"ukidssy" ,"norm":3,"zp":3631},
    'jpetromag (UKIDSS DR11)': {"bname":"ukidssj" ,"norm":3,"zp":3631},
    'hpetromag (UKIDSS DR11)': {"bname":"ukidssh" ,"norm":3,"zp":3631},
    'kpetromag (UKIDSS DR11)': {"bname":"ukidssk" ,"norm":3,"zp":3631},
    'targeted r mag'         : {"bname":"sdssr","norm":3,"zp":3631},
    'targeted g mag'         : {"bname":"sdssg","norm":3,"zp":3631},
    'targeted J mag'         : {"bname":"jband","norm":1,"zp":1631.00},
    'targeted H mag'         : {"bname":"hband","norm":1,"zp":1024.00},
    'targeted KS mag'        : {"bname":"kband","norm":1,"zp":675.70},
    'i1mag'                  : {"bname":"IRAC_ch1_total_response","norm":2,"zp":277.50},
    'i2mag'                  : {"bname":"IRAC_ch2_total_response","norm":2,"zp":179.50},
    'u (SDSS DR18)'          : {"bname":"sdssu","norm":3,"zp":3631.},
    'g (SDSS DR18)'          : {"bname":"sdssg","norm":3,"zp":3631.},
    'r (SDSS DR18)'          : {"bname":"sdssr","norm":3,"zp":3631.},
    'i (SDSS DR18)'          : {"bname":"sdssi","norm":3,"zp":3631.},
    'z (SDSS DR18)'          : {"bname":"sdssz","norm":3,"zp":3631.},
}

In [16]:
#Write the bandmag.dat file. 
cato = open("bandmag.dat","w")
cato.write("##\n")
cato.write("#Filter Name                   Norm       Zero Point\n")
cato.write("##\n")
for col in cols[::2]:
    filt = filters[col]
    cato.write("{:30s} {} {:18.3f}\n".format(filt['bname'], filt['norm'], filt['zp']))
cato.close()

In [17]:
subprocess.call("make single", shell=True)
subprocess.call("./single")

gfortran -ffixed-line-length-none -std=legacy  -o single single.f kca_agn_logred.f my_nnls_2.f


my_nnls_2.f:709:43:

  709 |       CALL H12(1,NPP1,NPP1+1,M,A(1,J),1,UP,DUMMY,1,1,0)
      |                                           1
-macosx_version_min has been renamed to -macos_version_min
ld: warning: ignoring duplicate libraries: '-lemutls_w', '-lgcc'


 Reading Filters and Setting Normalizations
 setting up           24  bands 
 corfac            1   5.6649139083033955        3.2916844579120621     
 corfac            2   5.8982764137547319        4.5427003364803635     
 corfac            3   4.6550266769077489        10.565650389253411     
 corfac            4   18.793446753997685        21.915713028010327     
 corfac            5  0.13505979666561796        1.0233348001986966     
 corfac            6   8.6256448618938678E-002   1.2534744183920743     
 corfac            7   6.0684997044566251E-002   1.6374135228454443     
 corfac            8   7.4343378607146474E-002   2.1363434728906494     
 corfac            9   55.017765649992683        1.0300409041343779     
 corfac           10   39.648644104962962        1.2474238038790908     
 corfac           11   22.149156435654518        1.6290120067928586     
 corfac           12   28.236630086972653        2.1985281057296850     
 corfac           13   7.8830776602735648      

 W0019-1046                    10
 W0116-0505                    15
 W0204-0506                    15
 W0220+0137                    13
 W0831+0140                    12
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DIVIDE_BY_ZERO IEEE_UNDERFLOW_FLAG


0

In [18]:
subprocess.call("make double", shell=True)
subprocess.call("./double")

gfortran -ffixed-line-length-none -std=legacy  -o double double.f kca_agn_logred.f my_nnls_2.f


my_nnls_2.f:709:43:

  709 |       CALL H12(1,NPP1,NPP1+1,M,A(1,J),1,UP,DUMMY,1,1,0)
      |                                           1
-macosx_version_min has been renamed to -macos_version_min
ld: warning: ignoring duplicate libraries: '-lemutls_w', '-lgcc'


 Reading Filters and Setting Normalizations
 setting up           24  bands 
 corfac            1   5.6649139083033955        3.2916844579120621     
 corfac            2   5.8982764137547319        4.5427003364803635     
 corfac            3   4.6550266769077489        10.565650389253411     
 corfac            4   18.793446753997685        21.915713028010327     
 corfac            5  0.13505979666561796        1.0233348001986966     
 corfac            6   8.6256448618938678E-002   1.2534744183920743     
 corfac            7   6.0684997044566251E-002   1.6374135228454443     
 corfac            8   7.4343378607146474E-002   2.1363434728906494     
 corfac            9   55.017765649992683        1.0300409041343779     
 corfac           10   39.648644104962962        1.2474238038790908     
 corfac           11   22.149156435654518        1.6290120067928586     
 corfac           12   28.236630086972653        2.1985281057296850     
 corfac           13   7.8830776602735648      

 W0019-1046                    10
 W0116-0505                    15
 W0204-0506                    15
 W0220+0137                    13
 W0831+0140                    12
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DIVIDE_BY_ZERO IEEE_UNDERFLOW_FLAG


0

In [19]:
os.chdir("f_stat")
subprocess.call("python compare.py", shell=True)
os.chdir("..")

In [20]:
bhd_c

<SkyCoord (ICRS): (ra, dec) in deg
    [(  4.8619973, -10.7759146), ( 19.0058816,  -5.0844703),
     ( 31.1921967,  -5.1113429), ( 35.2171736,   1.6198881),
     (127.9718723,   1.6696591)]>